# Project 2

In [45]:
from brian2 import *
import matplotlib.pyplot as plt
import random
from colour import Color
map_size = 100
global foodx, foody, food_count, bug_plot, food_plot, agg_sr_plot, agg_sl_plot,outbugx,outbugy
global outbugang,outfoodx,outfoody,outaggsrx,outaggsry,outaggslx,outaggsly, duration
global HN, food_array, outfoodcolors, colors, HopfieldOutput, current, outfoodcolors_rgb, food_arrays, ks
start_scope()
food_count = 0
foodx=50
foody=50
duration=2000 #1000
outbugx=np.zeros(int(duration/2))
outbugy=np.zeros(int(duration/2))
outbugang=np.zeros(int(duration/2))
outfoodx=np.zeros(int(duration/2))
outfoody=np.zeros(int(duration/2))
outaggsrx=np.zeros(int(duration/2))
outaggsry=np.zeros(int(duration/2))
outaggslx=np.zeros(int(duration/2))
outaggsly=np.zeros(int(duration/2))
outfoodcolors = []
outfoodcolors_rgb = []
red = Color("red")
colors = list(red.range_to(Color("green"),9))
HopfieldOutput = []
current = []
food_arrays = []
ks = []

In [46]:
import numpy as np

class HopfieldNetwork():

	def __init__(self,width,height):

		self.width = width

		self.height = height

		self.W = None # init with an empty W

		self.A = None # output

	def trainNetwork(self,X):
		# is is all the patterns wow
		# Train the network

		self.W = np.zeros((self.width * self.height, self.width * self.height))

		# sum outer product then subtract the identity matrix*number of weights? 
		outerprod = 0;
		for m in range(len(X)):
		    outerprod += (np.outer(X[m],(X[m])))
		#Add code here to update the weights   
		self.W = outerprod - len(X)*np.identity(self.height**2)

	def testNetwork(self,A):

		self.A = A

	def updateNetwork(self):

	    self.A=np.sign(np.dot(self.W,self.A))

In [47]:
# hopfield network 
HN = HopfieldNetwork(2,2)
X = np.zeros((2, 2 * 2))
X[0] = [1, 1, 1, 1]
X[1] = [-1,-1, -1, -1]
#X[2] = [-1,-1, 1, -1] #bad
HN.trainNetwork(X)

In [48]:
def generate_food(n=4):
    #Generate food with random attributes of size n
    food_array = []
    for i in range(n):
        if random.random() < 0.5:
            food_array.append(1)
        else:
            food_array.append(-1)
    return food_array

def convertHopfieldOutput(food_array,scale_factor=500):
    # Convert Hopfield output to stimulus  
    k1 = food_array.count(-1) * scale_factor
    k2 = food_array.count(1) * scale_factor
    
          
    
    return k1,k2

food_array = generate_food()
print(food_array)
HN.testNetwork(food_array)

[1, -1, -1, -1]


In [49]:
# Sensor neurons
a = 0.02
b = 0.25 #0.2
c = -65
#d = 0.5
d = 2

I0 = 900
tau_alpha=1.5*ms#1*ms
g_synpk=1.5
taupois=3*ms
g_synmaxval=(g_synpk/(tau_alpha/ms*exp(-1)))

sensor_eqs = '''
# equations for neurons
    dv/dt = ((0.04 * v**2 + 5 * v + 140 - u ) + (aggs *(0 - v)) + (cows *(0 - v)) + (aggsWTA *(-80 - v)) + (cowsWTA *(-80 - v)) + I)/ms : 1
    du/dt = (a * (b * v - u))/ms : 1
    x : 1
    y : 1
    x_disp : 1
    y_disp : 1
    foodxx : 1
    foodyy : 1
    mag :1
    HopfieldOutput: 1 
    I = (I0 + HopfieldOutput) / sqrt(((x-foodxx)**2+(y-foodyy)**2)): 1
    aggs : 1
    cows : 1
    dgsynpois/dt=-gsynpois/taupois :1
    
    
    dg_cowsWTA/dt = -cowsWTA/tau_alpha +z_cowsWTA/ms : 1
    dz_cowsWTA/dt= -z_cowsWTA/tau_alpha :1
    dg_aggsWTA/dt = -aggsWTA/tau_alpha +z_aggsWTA/ms : 1
    dz_aggsWTA/dt=- z_aggsWTA/tau_alpha :1
    aggsWTA : 1
    cowsWTA : 1
   
'''

sensor_reset = '''
    v = c
    u = u + d
'''

In [50]:
# right agg sensor
agg_sr = NeuronGroup(1, sensor_eqs, clock=Clock(0.2*ms), threshold = "v>=30", reset = sensor_reset,method='euler')
agg_sr.v = c
agg_sr.u = c*b
agg_sr.x_disp = 5
agg_sr.y_disp = 5
agg_sr.x = agg_sr.x_disp
agg_sr.y = agg_sr.y_disp
agg_sr.foodxx = foodx #need to get from Hopfield 
agg_sr.foodyy = foody
agg_sr.mag=1

# left agg sensor
agg_sl = NeuronGroup(1, sensor_eqs, clock=Clock(0.2*ms), threshold = "v>=30", reset = sensor_reset,method='euler')
agg_sl.v = c
agg_sl.u = c*b
agg_sl.x_disp = -5
agg_sl.y_disp = 5
agg_sl.x = agg_sl.x_disp
agg_sl.y = agg_sl.y_disp
agg_sl.foodxx = foodx
agg_sl.foodyy = foody
agg_sl.mag=1


# right cow sensor
cow_sr = NeuronGroup(1, sensor_eqs, clock=Clock(0.2*ms), threshold = "v>=30", reset = sensor_reset,method='euler')
cow_sr.v = c
cow_sr.u = c*b
cow_sr.x_disp = 5
cow_sr.y_disp = 5
cow_sr.x = cow_sr.x_disp
cow_sr.y = cow_sr.y_disp
cow_sr.foodxx = foodx
cow_sr.foodyy = foody
cow_sr.mag=1

# left cow sensor
cow_sl = NeuronGroup(1, sensor_eqs, clock=Clock(0.2*ms), threshold = "v>=30", reset = sensor_reset,method='euler')
cow_sl.v = c
cow_sl.u = c*b
cow_sl.x_disp = -5
cow_sl.y_disp = 5
cow_sl.x = cow_sl.x_disp
cow_sl.y = cow_sl.y_disp
cow_sl.foodxx = foodx
cow_sl.foodyy = foody
cow_sl.mag=1


# right bug motor neuron
sbr = NeuronGroup(1, sensor_eqs, clock=Clock(0.2*ms), threshold = "v>=30", reset = sensor_reset,method='euler')
sbr.v = c
sbr.u = c*b
sbr.foodxx = foodx
sbr.foodyy = foody
sbr.mag=0

# left bug motor neuron
sbl = NeuronGroup(1, sensor_eqs, clock=Clock(0.2*ms), threshold = "v>=30", reset = sensor_reset,method='euler')
sbl.v = c
sbl.u = c*b
sbl.foodxx = foodx
sbl.foodyy = foody
sbl.mag=0

In [51]:
# The virtual bug

taum = 3.5*ms
base_speed = 2
turn_rate = 5*Hz #if the turn rate is small then bug has to spin around more to get to the food 
alpha = 0.025 #0.05 #WANT ALPHA TO BE PRETTY SMALL OR ELSE THE SPEED GOES WILD AND MAKES IT HARD TO TURN
# alpha=0.01 leads it to just hover by the food, maybe it's a lover? 

bug_eqs = '''
#equations for movement here
    dx/dt = (alpha * speed * cos(angle))/ms : 1
    dy/dt = (alpha * speed * sin(angle))/ms : 1
    speed = ((motorl + motorr)/2) + base_speed : 1
    dangle/dt = (motorr - motorl) * turn_rate : 1
    dmotorl/dt = -motorl/taum : 1
    dmotorr/dt = -motorr/taum : 1
'''
#These are the equation  for the motor and speed

bug = NeuronGroup(1, bug_eqs, clock=Clock(0.2*ms),method='euler')
bug.motorl = 1 # 0 
bug.motorr = 1 # 0
bug.angle = pi/2
bug.x = 0
bug.y = 0


In [52]:
# Synapses (sensors communicate with bug motor)
w = 10
agg_syn_rr=Synapses(agg_sr, sbl, clock=Clock(0.2*ms), model='''
                g_synmax : 1
                dg_syn/dt = (-g_syn/tau_alpha) + z/ms : 1
                dz/dt = (-z/(tau_alpha/ms))/ms : 1
                aggs_post = g_syn :1 (summed)
                
                ''',
            on_pre='''
                z+= g_synmax
            ''')
agg_syn_rr.connect(i=[0],j=[0])
agg_syn_rr.g_synmax=g_synmaxval * 1.25

agg_syn_ll=Synapses(agg_sl, sbr, clock=Clock(0.2*ms), model='''
                g_synmax : 1
                dg_syn/dt = (-g_syn/tau_alpha) + z/ms : 1
                dz/dt = (-z/(tau_alpha/ms))/ms : 1
                aggs_post = g_syn :1 (summed)
                
                ''',
            on_pre='''
                z+= g_synmax
            ''')
agg_syn_ll.connect(i=[0],j=[0])
agg_syn_ll.g_synmax=g_synmaxval * 1.25

cow_syn_rr=Synapses(cow_sr, sbr, clock=Clock(0.2*ms), model='''
                g_synmax : 1
                dg_syn/dt = (-g_syn/tau_alpha) + z/ms : 1
                dz/dt = (-z/(tau_alpha/ms))/ms : 1
                cows_post = g_syn :1 (summed)
                ''',
            on_pre='''
                z+= g_synmax
            ''')
cow_syn_rr.connect(i=[0],j=[0])
cow_syn_rr.g_synmax=g_synmaxval

cow_syn_ll=Synapses(cow_sl, sbl, clock=Clock(0.2*ms), model='''
                g_synmax : 1
                dg_syn/dt = (-g_syn/tau_alpha) + z/ms : 1
                dz/dt = (-z/(tau_alpha/ms))/ms : 1
                cows_post = g_syn :1 (summed)
                
                ''',
            on_pre='''
                z+= g_synmax
            ''')
cow_syn_ll.connect(i=[0],j=[0])
cow_syn_ll.g_synmax=g_synmaxval

syn_r = Synapses(sbr, bug, clock=Clock(0.2*ms), on_pre='motorr += w')
syn_r.connect(i=[0],j=[0])
syn_l = Synapses(sbl, bug, clock=Clock(0.2*ms), on_pre='motorl += w')
syn_l.connect(i=[0],j=[0])

## connect for WTA LEFT
leftWTA1 = Synapses(cow_sl,agg_sl,clock=Clock(defaultclock.dt),model='''
                g_synmax:1
                ''',
            on_pre='''
                z_cowsWTA+= g_synmax
''')
leftWTA1.connect(i=[0], j=[0])  
leftWTA1.g_synmax=g_synmaxval

leftWTA2 = Synapses(agg_sl,cow_sl,clock=Clock(defaultclock.dt),model='''
                g_synmax:1
                ''',
            on_pre='''
                z_aggsWTA+= g_synmax
''')
leftWTA2.connect(i=[0], j=[0])  
leftWTA2.g_synmax=g_synmaxval

## connect for WTA RIGHT
rightWTA1 = Synapses(cow_sr,agg_sr,clock=Clock(defaultclock.dt),model='''
                g_synmax:1
                ''',
            on_pre='''
                z_cowsWTA+= g_synmax
''')
rightWTA1.connect(i=[0], j=[0])  
rightWTA1.g_synmax=g_synmaxval

rightWTA2 = Synapses(agg_sr,cow_sr,clock=Clock(defaultclock.dt),model='''
                g_synmax:1
                ''',
            on_pre='''
                z_aggsWTA+= g_synmax
''')
rightWTA2.connect(i=[0], j=[0])  
rightWTA2.g_synmax=g_synmaxval

In [53]:
@network_operation()
def update_positions():
    global foodx, foody, food_count
    global HN, food_array, food_arrays
    agg_sr.x = bug.x + agg_sr.x_disp*sin(bug.angle)+ agg_sr.y_disp*cos(bug.angle) 
    agg_sr.y = bug.y + - agg_sr.x_disp*cos(bug.angle) + agg_sr.y_disp*sin(bug.angle) 

    agg_sl.x = bug.x +  agg_sl.x_disp*sin(bug.angle)+agg_sl.y_disp*cos(bug.angle)
    agg_sl.y = bug.y  - agg_sl.x_disp*cos(bug.angle)+agg_sl.y_disp*sin(bug.angle) 
    
    cow_sr.x = agg_sr.x#bug.x + gsr.x_disp*sin(bug.angle)+ gsr.y_disp*cos(bug.angle)
    cow_sr.y = agg_sr.y#bug.y + - gsr.x_disp*cos(bug.angle) + gsr.y_disp*sin(bug.angle)

    cow_sl.x = agg_sl.x#bug.x +  gsl.x_disp*sin(bug.angle)+gsl.y_disp*cos(bug.angle)
    cow_sl.y = agg_sl.y#bug.y  - gsl.x_disp*cos(bug.angle)+gsl.y_disp*sin(bug.angle)

    # if bug catches food: 
    if (((bug.x-foodx)**2+(bug.y-foody)**2) < 25) or (((bug.x-foodx)**2+(bug.y-foody)**2) > 15000) :
        food_count += 1
        foodx = randint(-map_size+10, map_size-10)
        foody = randint(-map_size+10, map_size-10)
        # new hopfield is generated
        #food_array = generate_food()
        food_array = [-1,-1,-1,-1]
        HN.testNetwork(food_array)
        

    if (bug.x < -map_size):
        bug.x = -map_size
        bug.angle = pi - bug.angle
    if (bug.x > map_size):
        bug.x = map_size
        bug.angle = pi - bug.angle
    if (bug.y < -map_size):
        bug.y = -map_size
        bug.angle = -bug.angle
    if (bug.y > map_size):
        bug.y = map_size
        bug.angle = -bug.angle

    agg_sr.foodxx = foodx
    agg_sr.foodyy = foody
    agg_sl.foodxx = foodx
    agg_sl.foodyy = foody
    cow_sr.foodxx = foodx
    cow_sr.foodyy = foody
    cow_sl.foodxx = foodx
    cow_sl.foodyy = foody
    
    # Hopfield 
    HN.updateNetwork()
    cow_sr.HopfieldOutput, agg_sr.HopfieldOutput = convertHopfieldOutput(list(HN.A))
    cow_sl.HopfieldOutput, agg_sl.HopfieldOutput = convertHopfieldOutput(list(HN.A))

In [54]:
@network_operation(dt=2*ms)
def update_plot(t):
    global foodx, foody, bug_plot, food_plot, agg_sr_plot, agg_sl_plot,outbugx,outbugy,outbugang,outfoodx,outfoody,outaggsrx,outaggsry,outaggslx,outaggsly,outcowsrx,outcowsry,outcowslx,outcowsly
    global HN, food_array, food_color, outfoodcolors, colors, HopfieldOutput, current, outfoodcolors_rgb, food_arrays
    indx=int(.5*t/ms+1)
    #bug_plot[0].remove()
    #food_plot[0].remove()
    #sr_plot[0].remove()
    #sl_plot[0].remove()
    bug_x_coords = [bug.x, bug.x-4*cos(bug.angle), bug.x-8*cos(bug.angle)]    # ant-like body
    bug_y_coords = [bug.y, bug.y-4*sin(bug.angle), bug.y-8*sin(bug.angle)]
    
    outbugx[indx-1]=bug.x[0]
    outbugy[indx-1]=bug.y[0]
    outbugang[indx-1]=bug.angle[0]
    outfoodx[indx-1]=foodx
    outfoody[indx-1]=foody
    outaggsrx[indx-1]=agg_sr.x[0]
    outaggsry[indx-1]=agg_sr.y[0]
    outaggslx[indx-1]=agg_sl.x[0]
    outaggsly[indx-1]=agg_sl.y[0]
    outfoodcolors.append(colors[sum(food_array)+4].hex)
    trunc_rgb = [round(rgb, 2) for rgb in colors[sum(food_array)+4].rgb]
    outfoodcolors_rgb.append(trunc_rgb[0:2])
    HopfieldOutput.append(HN.A)
    current.append(convertHopfieldOutput(list(HN.A)))
    food_arrays.append(food_array)
    ks.append([cow_sr.HopfieldOutput, agg_sr.HopfieldOutput])
    
run(duration*ms,report='text')
np.save('outbugx',outbugx)
np.save('outbugy',outbugy)
np.save('outbugang',outbugang)
np.save('outfoodx',outfoodx)
np.save('outfoody',outfoody)
np.save('outaggsrx',outaggsrx)
np.save('outaggsry',outaggsry)
np.save('outaggslx',outaggslx)
np.save('outaggsly',outaggsly)
np.save('outfoodcolors', outfoodcolors)
np.save('HopfieldOutput',HopfieldOutput)
np.save('current',current)
np.save('outfoodcolors_rgb', outfoodcolors_rgb)
np.save('food_arrays', food_arrays)
np.save('ks', ks)

INFO       No numerical integration method specified for group 'synapses_7', using method 'exact' (took 0.03s). [brian2.stateupdaters.base.method_choice]
WARNING    Note that the synaptic pathway 'synapses_2_pre' will run on the clock of the group 'neurongroup_3' using a dt of 200. us. Either the Synapses object 'synapses_2' or the target 'neurongroup_1' (or both) are using a different dt. This might lead to unexpected results. In particular, all delays will be rounded to multiples of 200. us. If in doubt, try to ensure that 'neurongroup_3', 'synapses_2', and 'neurongroup_1' use the same dt. [brian2.synapses.synapses.synapses_dt_mismatch]
WARNING    Note that the synaptic pathway 'synapses_3_pre' will run on the clock of the group 'neurongroup_1' using a dt of 200. us. Either the Synapses object 'synapses_3' or the target 'neurongroup_3' (or both) are using a different dt. This might lead to unexpected results. In particular, all delays will be rounded to multiples of 200. us. If in do

Starting simulation at t=0. s for a duration of 2. s
3
1
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4
0
next
4


## show bug 

In [55]:
import numpy as np

import plotly.graph_objects as go

import numpy as np

import plotly.io as pio

pio.renderers.default='browser'

Ox=np.load('outbugx.npy')
Oy=np.load('outbugy.npy')
agg_srx=np.load('outaggsrx.npy')
agg_sry=np.load('outaggsry.npy')
agg_slx=np.load('outaggslx.npy')
agg_sly=np.load('outaggsly.npy')
food_color = np.load('outfoodcolors.npy')
HopfieldOutput = np.load('HopfieldOutput.npy')
current = np.load('current.npy')
outfoodcolors_rgb = np.load('outfoodcolors_rgb.npy')
food_arrays = np.load('food_arrays.npy')
ks = np.load('ks.npy')
Ba=np.load('outbugang.npy')
Fx=np.load('outfoodx.npy')
Fy=np.load('outfoody.npy')

timeframe = int(duration/2)
bug_x_coords=np.zeros((timeframe,3))
bug_y_coords=np.zeros((timeframe,3))



for i in range(0,timeframe):
           # Remove the last bug's position from the figure window
    bug_x_coords[i] = [Ox[i], Ox[i]-4*cos(Ba[i]), Ox[i]-8*cos(Ba[i])]
    bug_y_coords[i] = [Oy[i], Oy[i]-4*sin(Ba[i]), Oy[i]-8*sin(Ba[i])]

# Create figure
fig = go.Figure(
    data=[go.Scatter(x=Fx, y=Fy,
                     mode="lines",
                     line=dict(width=2, color="blue"))],
    
    
    layout=go.Layout(
        xaxis=dict(range=[-100, 100], autorange=False, zeroline=False),
        yaxis=dict(range=[-100, 100], autorange=False, zeroline=False),
        height=600,
        width=600,
        title_text="The Conscious Eater", hovermode="closest",
        updatemenus=[dict(type="buttons",
                          buttons=[dict(
                                        label="Play",
                                        method="animate",
                                        args = [None, {"frame": {"duration": 10, 
                                                                        "redraw": False},
                                                              "fromcurrent": True, 
                                                              "transition": {"duration": 0}}])])]),
    frames=[
        go.Frame(
        data=[go.Scatter(
            x=np.concatenate((bug_x_coords[k],[agg_srx[k], agg_slx[k]],[Fx[k]], [Fx[k]])),
            y=np.concatenate((bug_y_coords[k],[agg_sry[k], agg_sly[k]],[Fy[k]], [Fy[k]])),
            mode="markers+markers+markers+markers+markers+markers+text+text+text", # body body body sensor sensor food 
            text=['','','Hop: '+str(HopfieldOutput[k]),str(ks[k][0]),str(ks[k][1]),'Color: '+str(outfoodcolors_rgb[k]),'Food input Array' + str(food_arrays[k])],
            textposition=['top left','top left','top left', 'top left','top right','top center', 'bottom center'],
            marker=dict(color=["black","black","black","blue","blue",outfoodcolors[k]], size=[10,10,10,8,8,20]))])
        for k in range(1,timeframe)]
)
fig.show()

In [14]:
f = [-1,1,-1,-1]
abs(f[0])

1